In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/ds-bowl-labels/__output__.json
/kaggle/input/ds-bowl-labels/labels_created_ini
/kaggle/input/ds-bowl-labels/custom.css
/kaggle/input/ds-bowl-labels/__results__.html
/kaggle/input/ds-bowl-labels/df_counter
/kaggle/input/ds-bowl-labels/__notebook__.ipynb
/kaggle/input/ds-bowl-labels/df_counter_ini
/kaggle/input/data-science-bowl-2019/sample_submission.csv
/kaggle/input/data-science-bowl-2019/specs.csv
/kaggle/input/data-science-bowl-2019/train_labels.csv
/kaggle/input/data-science-bowl-2019/train.csv
/kaggle/input/data-science-bowl-2019/test.csv
/kaggle/input/temp-ds-bowl-2019/labels_created_ini
/kaggle/input/temp-ds-bowl-2019/df_counter
/kaggle/input/temp-ds-bowl-2019/df_counter_ini


In [2]:
import IPython

def display(*dfs):
    for df in dfs:
        IPython.display.display(df)

# display(df1, df2)

In [3]:
## Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [4]:
%time df = pd.read_csv('/kaggle/input/data-science-bowl-2019/train.csv', engine='c')
labels = pd.read_csv('/kaggle/input/data-science-bowl-2019/train_labels.csv')

CPU times: user 1min 12s, sys: 16.9 s, total: 1min 29s
Wall time: 1min 29s


In [5]:
df = reduce_mem_usage(df)
labels = reduce_mem_usage(labels)

Mem. usage decreased to 778.73 Mb (18.2% reduction)
Mem. usage decreased to  0.49 Mb (48.2% reduction)


In [6]:
df.shape, labels.shape

((11341042, 11), (17690, 7))

In [7]:
# # Format and make date / hour features
df['timestamp'] = pd.to_datetime(df['timestamp'])
# df['date'] = df['timestamp'].dt.date
# df['hour'] = df['timestamp'].dt.hour
# df['weekday_name'] = df['timestamp'].dt.weekday_name
# # Same for test
# # test['timestamp'] = pd.to_datetime(test['timestamp'])
# # test['date'] = test['timestamp'].dt.date
# # test['hour'] = test['timestamp'].dt.hour
# # test['weekday_name'] = test['timestamp'].dt.weekday_name

In [8]:
import gc
gc.collect()

11

In [9]:
df_test = pd.read_csv('/kaggle/input/data-science-bowl-2019/test.csv')

In [10]:
df_test.shape

(1156414, 11)

In [11]:
sample_subm = pd.read_csv('/kaggle/input/data-science-bowl-2019/sample_submission.csv')
sample_subm.head()

,installation_id,accuracy_group
0,00abaee7,3
1,01242218,3
2,017c5718,3
3,01a44906,3
4,01bc6cb6,3


In [12]:
specs = pd.read_csv('/kaggle/input/data-science-bowl-2019/specs.csv')
specs.head()

,event_id,info,args
0,2b9272f4,The end of system-initiated feedback (Correct)...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
1,df4fe8b6,The end of system-initiated feedback (Incorrec...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
2,3babcb9b,The end of system-initiated instruction event ...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
3,7f0836bf,The end of system-initiated instruction event ...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
4,ab3136ba,The end of system-initiated instruction event ...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."


In [13]:
# df_train = df.copy()
# df = df_test

In [14]:
# df = df_train

# Create labels

In [15]:
def create_label_all(df, labels=None):
#     id_unique = labels.installation_id.unique()
#     print('Numer of unique installation_id:', id_unique.shape[0])
    
    if labels is not None:
        # labels_part = labels[labels.installation_id.isin(id_unique[:n])]
        labels_part = labels.copy()
        
    # df_ids = df[df.installation_id.isin(id_unique[:n])].reset_index()
    df_ids = df.reset_index(drop=True)
    assert df_ids.shape[0] == df.shape[0]
    
    extracted_event = df_ids[df_ids.type == 'Assessment']
    print('# unique ids ini:', df_ids.installation_id.unique().shape[0])
    print('# unique ids in assessments', extracted_event.installation_id.unique().shape[0])
    extracted_event['correct'] = np.where(extracted_event.event_data.str.find('"correct":true')!=-1, True, False)
    # print('Number of assessments with "correct" in event_data:', extracted_event.shape)

    df_ids.loc[:, 'correct'] = extracted_event.correct
    #df_ids.update()
    del extracted_event
    
    index_for_correct = ~df_ids.correct.isna()
#     df_ids[index_for_correct]

    df_counter = df_ids[index_for_correct]
    df_counter_wt_BM = df_counter[(df_counter.event_code == 4100) & (df_counter.title.str.find('Bird Measurer')==-1)]
    df_counter_BM = df_counter[(df_counter.event_code == 4110) & (df_counter.title.str.find('Bird Measurer')!=-1)]

    df_counter_ini = df_counter_wt_BM.append(df_counter_BM)
    df_counter = df_counter_ini.groupby('game_session', as_index=False).agg({'correct': 'sum',
                                                         'event_code': 'count',
                                                         'installation_id': lambda x: x.iloc[0],
                                                         'title': lambda x: x.iloc[0]})

    df_counter['accuracy'] = df_counter.correct/df_counter.event_code
    if labels is not None:
        df_check = df_counter.merge(labels_part, on='game_session', how='left')
    
    return df_counter_ini, df_counter, df_check

# df_counter_ini, df_counter, df_check = create_label_all(df, labels)

In [16]:
download_or_not = False

In [17]:
# df_counter_dw = df_counter.copy()
# df_counter_ini_dw = df_counter_ini.copy()

In [18]:
if download_or_not:
    print('Reading feathers')
    df_counter_ini = pd.read_feather('/kaggle/input/temp-ds-bowl-2019/df_counter_ini')
    df_counter = pd.read_feather('/kaggle/input/temp-ds-bowl-2019/df_counter')
else: 
    print('Calculating')
    df_counter_ini, df_counter, df_check = create_label_all(df, labels)
    
    print('Writing to feather')
    df_counter.correct = df_counter.correct.astype(int)
    df_counter_ini.reset_index().to_feather('df_counter_ini')
    df_counter.reset_index().to_feather('df_counter')
    

Calculating
# unique ids ini: 17000
# unique ids in assessments 4242


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


Writing to feather


In [19]:
# import json 
# extracted_event_data = pd.io.json.json_normalize(df_ids.event_data.apply(json.loads))
# print(extracted_event_data.shape)

In [20]:
'New lables:', df_counter.shape[0] - labels.shape[0]

('New lables:', 0)

In [21]:
try:
    df_error = df_check[(np.abs(df_check.accuracy_x-df_check.accuracy_y) > 0.001) & 
                         (~df_check.accuracy_y.isna())]
    print('Number of error calculated accuracy:', df_error.shape[0])
    #df_error
except:
    pass

Number of error calculated accuracy: 0


# Data study

## Check ids which never took assessments

In [22]:
print('# unique ids with assessment:', df.installation_id.unique().shape)
print('# unique ids in train:', df_counter.installation_id.unique().shape)
print('# unique ids in train_labels:', labels.installation_id.unique().shape)

# unique ids with assessment: (17000,)
# unique ids in train: (3614,)
# unique ids in train_labels: (3614,)


## Create initial labels (like in train_labels file)

In [23]:
display(labels, df_counter, df_counter_ini)

,game_session,installation_id,title,num_correct,num_incorrect,accuracy,accuracy_group
0,6bdf9623adc94d89,0006a69f,Mushroom Sorter (Assessment),1,0,1.000000,3
1,77b8ee947eb84b4e,0006a69f,Bird Measurer (Assessment),0,11,0.000000,0
2,901acc108f55a5a1,0006a69f,Mushroom Sorter (Assessment),1,0,1.000000,3
3,9501794defd84e4d,0006a69f,Mushroom Sorter (Assessment),1,1,0.500000,2
4,a9ef3ecb3d1acc6a,0006a69f,Bird Measurer (Assessment),1,0,1.000000,3
...,...,...,...,...,...,...,...
17685,c996482b11d149dd,ffc90c32,Bird Measurer (Assessment),1,0,1.000000,3
17686,b05a02b52d5c1f4c,ffd2871d,Cauldron Filler (Assessment),1,0,1.000000,3
17687,5448d652309a6324,ffeb0b1b,Cauldron Filler (Assessment),1,2,0.333252,1
17688,a6885ab824fbc32c,ffeb0b1b,Mushroom Sorter (Assessment),0,1,0.000000,0


,game_session,correct,event_code,installation_id,title,accuracy
0,00097cda27afb726,1,1,01bdd720,Mushroom Sorter (Assessment),1
1,000f68cff32664ef,0,6,3f0dca37,Chest Sorter (Assessment),0
2,0014403daadf67aa,1,4,29d1aaee,Bird Measurer (Assessment),0.25
3,0014daa1d3e26eb2,1,3,55fdf49f,Mushroom Sorter (Assessment),0.333333
4,001c49e9e9968dbe,1,2,19d4b097,Bird Measurer (Assessment),0.5
...,...,...,...,...,...,...
17685,ffe9dd5aa0bf9231,1,3,53858b6a,Cauldron Filler (Assessment),0.333333
17686,fff20fa828fb1a94,1,1,5fb6ef75,Cart Balancer (Assessment),1
17687,fff441b089d7b1ef,1,1,54d91d26,Cauldron Filler (Assessment),1
17688,fff53215178ede64,1,1,4db1e993,Bird Measurer (Assessment),1


,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world,correct
2228,25fa8af4,901acc108f55a5a1,2019-08-06 05:22:32.357000+00:00,"{""correct"":true,""stumps"":[1,2,4],""event_count""...",0006a69f,44,4100,31011,Mushroom Sorter (Assessment),Assessment,TREETOPCITY,True
2797,25fa8af4,6bdf9623adc94d89,2019-08-06 05:38:08.036000+00:00,"{""correct"":true,""stumps"":[1,2,4],""event_count""...",0006a69f,30,4100,18026,Mushroom Sorter (Assessment),Assessment,TREETOPCITY,True
3725,25fa8af4,9501794defd84e4d,2019-08-06 20:35:12.290000+00:00,"{""correct"":false,""stumps"":[3,2,5],""event_count...",0006a69f,29,4100,18484,Mushroom Sorter (Assessment),Assessment,TREETOPCITY,False
3734,25fa8af4,9501794defd84e4d,2019-08-06 20:35:16.846000+00:00,"{""correct"":true,""stumps"":[2,3,5],""event_count""...",0006a69f,38,4100,23043,Mushroom Sorter (Assessment),Assessment,TREETOPCITY,True
6900,392e14df,197a373a77101924,2019-09-14 15:35:54.361000+00:00,"{""buckets"":[0,0,0],""correct"":true,""buckets_pla...",0006c192,20,4100,12635,Cauldron Filler (Assessment),Assessment,MAGMAPEAK,True
...,...,...,...,...,...,...,...,...,...,...,...,...
11322443,17113b36,967f050913e025e7,2019-08-30 13:17:54.525000+00:00,"{""correct"":false,""caterpillars"":[2,2,2],""event...",ff9305d7,4,4110,3844,Bird Measurer (Assessment),Assessment,TREETOPCITY,False
11322462,17113b36,967f050913e025e7,2019-08-30 13:18:12.198000+00:00,"{""correct"":false,""caterpillars"":[3,8,5],""event...",ff9305d7,23,4110,21529,Bird Measurer (Assessment),Assessment,TREETOPCITY,False
11322468,17113b36,967f050913e025e7,2019-08-30 13:18:15.871000+00:00,"{""correct"":false,""caterpillars"":[3,8,5],""event...",ff9305d7,29,4110,25197,Bird Measurer (Assessment),Assessment,TREETOPCITY,False
11322479,17113b36,967f050913e025e7,2019-08-30 13:18:24.573000+00:00,"{""correct"":true,""caterpillars"":[4,8,5],""event_...",ff9305d7,40,4110,33897,Bird Measurer (Assessment),Assessment,TREETOPCITY,True


In [24]:
df_counter['num_incorrect'] = df_counter.event_code - df_counter.correct
df_counter.rename(columns={'correct': 'num_correct', 'event_code': 'total_attempts'}, inplace=True)
df_counter.head()

,game_session,num_correct,total_attempts,installation_id,title,accuracy,num_incorrect
0,00097cda27afb726,1,1,01bdd720,Mushroom Sorter (Assessment),1,0
1,000f68cff32664ef,0,6,3f0dca37,Chest Sorter (Assessment),0,6
2,0014403daadf67aa,1,4,29d1aaee,Bird Measurer (Assessment),0.25,3
3,0014daa1d3e26eb2,1,3,55fdf49f,Mushroom Sorter (Assessment),0.333333,2
4,001c49e9e9968dbe,1,2,19d4b097,Bird Measurer (Assessment),0.5,1


In [25]:
df_counter.reset_index(inplace=True)

In [26]:
acc_group_map = {1: 3, 0.5: 2, 0: 0}
df_counter['accuracy_group'] = df_counter.accuracy.apply(lambda x: acc_group_map[x] if x >=0.5 or x ==0 else 1)

In [27]:
temp = df_counter.merge(labels[['accuracy', 'accuracy_group', 'game_session']], how='left', on='game_session')
temp[temp.accuracy_group_x != temp.accuracy_group_y]

,index,game_session,num_correct,total_attempts,installation_id,title,accuracy_x,num_incorrect,accuracy_group_x,accuracy_y,accuracy_group_y


In [28]:
columns_use = list(labels.columns)
columns_use.insert(5, 'total_attempts')
df_counter = df_counter[columns_use]
display(temp, labels)

,index,game_session,num_correct,total_attempts,installation_id,title,accuracy_x,num_incorrect,accuracy_group_x,accuracy_y,accuracy_group_y
0,0,00097cda27afb726,1,1,01bdd720,Mushroom Sorter (Assessment),1,0,3,1.000000,3
1,1,000f68cff32664ef,0,6,3f0dca37,Chest Sorter (Assessment),0,6,0,0.000000,0
2,2,0014403daadf67aa,1,4,29d1aaee,Bird Measurer (Assessment),0.25,3,1,0.250000,1
3,3,0014daa1d3e26eb2,1,3,55fdf49f,Mushroom Sorter (Assessment),0.333333,2,1,0.333252,1
4,4,001c49e9e9968dbe,1,2,19d4b097,Bird Measurer (Assessment),0.5,1,2,0.500000,2
...,...,...,...,...,...,...,...,...,...,...,...
17685,17685,ffe9dd5aa0bf9231,1,3,53858b6a,Cauldron Filler (Assessment),0.333333,2,1,0.333252,1
17686,17686,fff20fa828fb1a94,1,1,5fb6ef75,Cart Balancer (Assessment),1,0,3,1.000000,3
17687,17687,fff441b089d7b1ef,1,1,54d91d26,Cauldron Filler (Assessment),1,0,3,1.000000,3
17688,17688,fff53215178ede64,1,1,4db1e993,Bird Measurer (Assessment),1,0,3,1.000000,3


,game_session,installation_id,title,num_correct,num_incorrect,accuracy,accuracy_group
0,6bdf9623adc94d89,0006a69f,Mushroom Sorter (Assessment),1,0,1.000000,3
1,77b8ee947eb84b4e,0006a69f,Bird Measurer (Assessment),0,11,0.000000,0
2,901acc108f55a5a1,0006a69f,Mushroom Sorter (Assessment),1,0,1.000000,3
3,9501794defd84e4d,0006a69f,Mushroom Sorter (Assessment),1,1,0.500000,2
4,a9ef3ecb3d1acc6a,0006a69f,Bird Measurer (Assessment),1,0,1.000000,3
...,...,...,...,...,...,...,...
17685,c996482b11d149dd,ffc90c32,Bird Measurer (Assessment),1,0,1.000000,3
17686,b05a02b52d5c1f4c,ffd2871d,Cauldron Filler (Assessment),1,0,1.000000,3
17687,5448d652309a6324,ffeb0b1b,Cauldron Filler (Assessment),1,2,0.333252,1
17688,a6885ab824fbc32c,ffeb0b1b,Mushroom Sorter (Assessment),0,1,0.000000,0


### How is samples looks like ?

In [29]:
def sequence_correct_attempt_by_game_session(df_counter_ini=df_counter_ini):
    df_counter_ini['correct_str'] = df_counter_ini.correct.astype(int).astype(str)
    to_add = df_counter_ini.groupby('game_session').agg({'correct_str': 'sum'})
    return to_add

In [30]:
#df_counter.reset_index(inplace=True)
to_add = sequence_correct_attempt_by_game_session(df_counter_ini)
assert to_add.shape[0], df_counter_download.shape[0]

df_counter = df_counter.merge(to_add, how='left', on='game_session')
df_counter

,game_session,installation_id,title,num_correct,num_incorrect,total_attempts,accuracy,accuracy_group,correct_str
0,00097cda27afb726,01bdd720,Mushroom Sorter (Assessment),1,0,1,1,3,1
1,000f68cff32664ef,3f0dca37,Chest Sorter (Assessment),0,6,6,0,0,000000
2,0014403daadf67aa,29d1aaee,Bird Measurer (Assessment),1,3,4,0.25,1,0001
3,0014daa1d3e26eb2,55fdf49f,Mushroom Sorter (Assessment),1,2,3,0.333333,1,001
4,001c49e9e9968dbe,19d4b097,Bird Measurer (Assessment),1,1,2,0.5,2,01
...,...,...,...,...,...,...,...,...,...
17685,ffe9dd5aa0bf9231,53858b6a,Cauldron Filler (Assessment),1,2,3,0.333333,1,001
17686,fff20fa828fb1a94,5fb6ef75,Cart Balancer (Assessment),1,0,1,1,3,1
17687,fff441b089d7b1ef,54d91d26,Cauldron Filler (Assessment),1,0,1,1,3,1
17688,fff53215178ede64,4db1e993,Bird Measurer (Assessment),1,0,1,1,3,1


In [31]:
import re
def find_unnormal_sequences(df):
    template = r'^[01]0*$' # start with 0or1 and then only 0s
    df.correct_str = df_counter.correct_str.str[::-1] # reverse string; now last attempt is on 0 index
    #print(df.head())
    return df[~df.correct_str.str.contains(template, regex=True)].index

In [32]:
idx = find_unnormal_sequences(df_counter)
df_counter.loc[idx]

,game_session,installation_id,title,num_correct,num_incorrect,total_attempts,accuracy,accuracy_group,correct_str


# Write new labels

In [33]:
df_counter.to_feather('labels_created_ini')